Import relevant packages

In [1]:
import glob
import pandas as pd
import tabula
from urllib.request import urlretrieve

Define functions for wrangling raw data into processable format

In [2]:
def add_year_column(tableidx):  
  table = pd.DataFrame(TABLES[landuse_tables[tableidx]])
  table['Year'] = table.iloc[:,0].str.strip('Estimate')
  estimates = table[table.index % 2 == 0]
  return estimates

def pdf_df(dataframe_list):  
  dataframe_df = dataframe_list[2]
  dataframe_df.iloc[1,0] = 'Year'
  dataframe_acres = dataframe_df.iloc[1:3,:].T
  dataframe_acres2 = dataframe_acres.iloc[1:]
  dataframe_acres2.columns = dataframe_acres.iloc[0,:]
  dataframe_acres2['Year'] = dataframe_acres2['Year'].astype('int')
  dataframe_acres2['Corn'] = [float(str(i).replace(",", "")) for i in dataframe_acres2['Corn']]
  dataframe_acres2['Corn (Thousands of acres)'] = dataframe_acres2['Corn'] /1000
  dataframe_acres2.reset_index(drop=True, inplace=True)
  return dataframe_acres2


def relevant_series_selection():
  seriesindex1 = int(input('Provide the column index of the first series: '))
  seriesindex2 = int(input('Provide the column index of the second series: '))
  relevant_table = estimates.iloc[:,[seriesindex1,seriesindex2]]
  series1_name = estimates.iloc[:,seriesindex1].name
  series2_name = estimates.iloc[:,seriesindex2].name
  relevant_table.reset_index(drop=True,inplace=True)
  relevant_table[series1_name] = relevant_table[series1_name].astype('float')
  relevant_table[series2_name] = relevant_table[series2_name].astype('int')
  relevant_table = relevant_table[[series2_name,series1_name]]

  return relevant_table

def observed_change_timeperiod(dataframe):
  differences = []
  years = []
  yearly_diff = []
  for row in dataframe.itertuples():
    if row[0] == min(range(len(dataframe))):
      prior_row = row
      continue
    else:
      differences.append(row[2] - prior_row[2])
      years.append(int(row[1]) - int(prior_row[1]))
      prior_row = row

  for diff,year in zip(differences,years):
    for yr in range(year-1):
      yearly_diff.append(diff/year)
  return yearly_diff

def create_missing_df(dataframe, series1, series2):
  miss_years = []
  for start in series1:
    for year in range(len(series1)):
      if (start + year) == start:
        continue
      elif (start + year) in list(series1) or (start + year) > max(series1):
        break
      else:
        miss_years.append(start + year)

  missing_df = pd.DataFrame()
  missing_df[series1.name] = miss_years
  missing_df[series2.name] = 0
  missing_df['Differences'] = observed_change_timeperiod(dataframe)
  return missing_df

def create_missing_practices(dataframe, series1, series2):
  miss_years = []
  for start in no_till_df['Year']:
    for year in range(len(no_till_df[no_till_df['Year'] < 2008])):
      if (start + year) == start:
        continue
      elif (start + year) in list(no_till_df['Year']) or (start + year) > max(no_till_df['Year']):
        break
      else:
        miss_years.append(start + year)

  missing_df = pd.DataFrame()
  missing_df[series1.name] = miss_years
  missing_df[series2.name] = 0
  missing_df['Differences'] = observed_change_timeperiod(dataframe)
  return missing_df

def parse_table(dataframe,miss_df): 
  dataframe['differences'] = 0
  relevant_table_parsed = dataframe.append(miss_df)
  relevant_table_parsed.sort_values(by='Year',inplace=True)
  relevant_table_parsed.reset_index(drop=True,inplace=True)
  return relevant_table_parsed

def calc_total_land_change(dataframe):
  TotalLand = []
  for row in dataframe.itertuples():
    if row[3] == 0:
      TotalLand.append(row[2])
      prior_row = row[2]
    else:
      TotalLand.append((prior_row + row[4]))
      prior_row = prior_row + row[4]
  
  return TotalLand

def fert_df_creation(df):
    IN_fert = df.iloc[[1,12],:]
    IN_fert = IN_fert.T
    IN_fert.iloc[0,0] = 'Year'
    IN_fert.iloc[0,1] = input('What fertilizer type? ')
    IN_fert.columns = IN_fert.iloc[0,:]
    IN_fert = IN_fert.iloc[1:]
    IN_fert.reset_index(drop=True,inplace=True)
    return IN_fert


Retrieve landuse data from USDA website

In [3]:
url = 'https://www.nrcs.usda.gov/Internet/NRCS_RCA/csv/nri_download_in.csv'

In [4]:
urlretrieve(url,'IN_landuse.csv')

('IN_landuse.csv', <http.client.HTTPMessage at 0x7f8840b99070>)

Iterate through each line of the USDA file and assign it to the tableslist

In [5]:
fileList = []
table = []
with open('IN_landuse.csv', 'rb') as file:
  for line in file:
    if len(line) < 6:
      if table != []:
        fileList.append(table)
        table = []
      continue
    else:
      table.append(line)

In [6]:
tableslist = fileList[1:23]

Create a list of tables as dicts from the tables in tableslist and strip any formating charcters from the string of the table name, from table_list_dict and table_names lists create accessable dictionary TABLES where the keys are the names of the tables and the values are dictionaries containing all of the relevant landuse data. 

In [7]:
table_names = []
for table in tableslist:
  if tableslist.index(table) == 0:
    table_names.append(str(table[1]).strip("b'").strip('\\r\\n'))
  elif (tableslist.index(table))%2 == 0:
    table_names.append(str(table[0]).strip("b'").strip('\\r\\n'))

In [8]:
table_raw = [table for table in tableslist if tableslist.index(table)%2 != 0]

In [9]:
list_table_dict = []
for table in table_raw:
  table_dict = {}
  for row in table:
    listrow = str(row).split(',')
    striprow = []
    for element in listrow:
      striprow.append(element.strip("b'").strip("\\r\\n"))
    column_name = striprow[0]
    if striprow[-1] == '':
      values = striprow[1:-1]
    else:
      values = striprow[1:]
    table_dict[column_name] = values
  list_table_dict.append(table_dict)


In [10]:
TABLES = {}
for name, table in zip(table_names, list_table_dict):
  TABLES[name] = table

In [11]:
landuse_tables = [key for key in TABLES.keys()]

In [12]:
landuse_tables

['Total Surface Area by Land Cover/Use',
 'Non-Federal Rural Land by Land Cover/Use',
 'Prime Farmland by Land Cover/Use',
 'Indiana Cropland Use',
 'Indiana  Erosio',
 'Indiana  Non-Federal Grazing Land',
 'Indiana  Non-Federal Forest Land',
 'Indiana  Developed Land',
 'Sources of Developed Land',
 'Indiana  Wetlands',
 'Palustrine and Estuarine Wetlands on Water Areas and Non-Federal Land by Land Cover/Use']

Call the table at index position 7, which contains IN landuse by development level, in the add_year_column function to strip only the integer of the year from the index.

In [13]:
estimates = add_year_column(int(input('Enter the desired table index: ')))
estimates.columns

Enter the desired table index: 7


Index(['Developed Land Type', 'Large Urban and Built-Up Areas ',
       'Small Built-Up Areas ', 'Rural Transportation ',
       'Total Developed Land ', 'Year'],
      dtype='object')

create relevant table from the appropriate series of landuse data, index positions 4 and 5.

In [14]:
relevant_table = relevant_series_selection()

Provide the column index of the first series: 4
Provide the column index of the second series: 5


<ipython-input-2-1ebbe04b62a3>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_table[series1_name] = relevant_table[series1_name].astype('float')
<ipython-input-2-1ebbe04b62a3>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_table[series2_name] = relevant_table[series2_name].astype('int')


Create a new dataframe (Dev_Land_Final), by calling the create_missing_df and parse_table functions on the previously defined relevant table

In [15]:
missing_df = create_missing_df(relevant_table,relevant_table.iloc[:,0],relevant_table.iloc[:,1])

In [16]:
relevant_table_parsed = parse_table(relevant_table,missing_df)

In [17]:
relevant_table_parsed['Total Dev Land (Thousands of acres)'] = calc_total_land_change(relevant_table_parsed)
Final_table = relevant_table_parsed[['Year','Total Dev Land (Thousands of acres)']]
Dev_Land_Final = Final_table

Repeat the above steps on the table in the TABLES dictionary at index position 1

In [18]:
estimates = add_year_column(int(input('Enter the desired table index: ')))
estimates.columns

Enter the desired table index: 1


Index(['Land Cover/Use', 'Cropland ', 'CRP Land ', 'Pastureland ',
       'Rangeland ', 'Forest Land ', 'Other Rural Land ',
       'Total Non-Federal Rural Land ', 'Year'],
      dtype='object')

create relevant_table dataframe from the series at positions 1 and 8. And call the create_missing_df and parse_table functions on it to produce Crop_Land_Final table.

In [19]:
relevant_table = relevant_series_selection()

Provide the column index of the first series: 1
Provide the column index of the second series: 8


<ipython-input-2-1ebbe04b62a3>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_table[series1_name] = relevant_table[series1_name].astype('float')
<ipython-input-2-1ebbe04b62a3>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_table[series2_name] = relevant_table[series2_name].astype('int')


In [20]:
missing_df = create_missing_df(relevant_table,relevant_table.iloc[:,0],relevant_table.iloc[:,1])

In [21]:
relevant_table_parsed = parse_table(relevant_table,missing_df)

In [22]:
relevant_table_parsed['Total Crop Land (Thousands of acres)'] = calc_total_land_change(relevant_table_parsed)
Final_table = relevant_table_parsed[['Year','Total Crop Land (Thousands of acres)']]
Crop_Land_Final = Final_table

Create the final_landuse table by merging the Dev_Land_Final dataframe to the Crop_Land_Final dataframe on year.

In [23]:
final_landuse = Crop_Land_Final.merge(Dev_Land_Final)

Create the crop_emissions_final dataframe using glob to process all the tables from the csv files in emissions_path.

In [24]:
emissions_path = '/Users/coleromanyk/Downloads/Chapter Text 2/Ch 5 - Agriculture/*.*'

In [25]:
file_list = []
table_list = []
for file in glob.glob(emissions_path):
    file_list.append(str(file))
    table = pd.read_csv(file, encoding= 'unicode_escape').T
    table_idx = table.index
    table_name = table_idx[0].split(':  ')[1]
    table_list.append(table_name)

In [26]:
GHG_tables = {name:file for name,file in zip(table_list,file_list)}

In [27]:
ghgtables = list(GHG_tables.keys())

In [28]:
corn_files = ['Agricultural Crop Production (kt of Product)','CH4, N2O, CO, and NOx Emissions from Field Burning of Agricultural Residues (kt)','Direct N2O Emissions from Agricultural Soils by Land Use Type and N Input Type (MMT CO2 Eq.)','Greenhouse Gas Emission Ratios and Conversion Factors','Quantitative Uncertainty Estimates of N2O Emissions from Agricultural Soil Management in 2018 (MMT CO2 Eq. and Percent)','Emissions from Agriculture (kt)','N2O Emissions from Agricultural Soils (MMT CO2 Eq.)','N2O Emissions from Agricultural Soils (kt)','Emissions from Agriculture (MMT CO2 Eq.)']

In [29]:
raw_table = pd.read_csv(GHG_tables[str(corn_files[8])])

In [30]:
raw_table = raw_table.iloc[1:,1:]
raw_table.iloc[0,0] = 'Year'

In [31]:
raw_table2 = raw_table.T

In [32]:
raw_table2.reset_index(drop=True,inplace=True)
raw_table2.columns = raw_table2.iloc[0,:]

In [33]:
raw_table2 = raw_table2.iloc[1:]

In [34]:
crop_emissions = raw_table2.loc[:,['Year','CO2']]
crop_emissions['N2O'] = raw_table2.loc[:,'N2O'] - raw_table2.iloc[:,11]
crop_emissions['CH4'] = raw_table2.iloc[:,8]
crop_emissions['Total (MMT CO2 Eq.)'] = crop_emissions.loc[:,'CO2'] + crop_emissions.loc[:,'N2O'] + crop_emissions.loc[:,'CH4'] 

In [35]:
crop_emissions_final = crop_emissions[['Year','Total (MMT CO2 Eq.)']]

Create the emissions_landuse dataframe by merging the dataframes containing the emissions data and the landuse data merging on year.

In [36]:
emissions_landuse = crop_emissions_final.merge(final_landuse)

Creat the IN_fert_final dataframe from the historical fertilzer data for IN by selecting the data from the nitrogen, phosphate, potash fertilizer use for corn sheets in the fertilizeruse workbook merging on on year.

In [37]:
fert_path = '/Users/coleromanyk/Downloads/fertilizeruse.xls'

In [38]:
corn_tables = ['Table10','Table12','Table14']

In [39]:
raw_nitrogen = pd.read_excel(fert_path,sheet_name=corn_tables[0])

In [40]:
IN_N = fert_df_creation(raw_nitrogen)

What fertilizer type? N


In [41]:
raw_phosphate = pd.read_excel(fert_path,sheet_name=corn_tables[1])

In [42]:
IN_PH = fert_df_creation(raw_phosphate)

What fertilizer type? PO4


In [43]:
raw_potash = pd.read_excel(fert_path,sheet_name=corn_tables[2])

In [44]:
IN_PO = fert_df_creation(raw_potash)

What fertilizer type? Potash


In [45]:
fert1 = IN_N.merge(IN_PH)
IN_fert_final = fert1.merge(IN_PO)

In [46]:
IN_fert_final.fillna(round(IN_fert_final.rolling(10,min_periods=1).mean(),),inplace=True)

Combine the fertilizer use data for IN with the US emissions and IN landuse data merging on year.

In [47]:
Emissions_landuse_fertilizer = emissions_landuse.merge(IN_fert_final)

Download and read the appropriate production practice data from the in.gov site. Then format the dataframes appropriately prior to merging on year.

In [48]:
pdf = 'https://www.in.gov/isda/files/No-Till-Trends-1990-2019-Statewide.pdf'
urlretrieve(pdf,'no_till.pdf')
no_till = tabula.read_pdf('no_till.pdf', lattice=True, multiple_tables=True)

In [49]:
pdf = 'https://www.in.gov/isda/files/Cover-Crop-Trends-2011-2019-Statewide.pdf'
urlretrieve(pdf,'cover_crop.pdf')
cover_crop = tabula.read_pdf('cover_crop.pdf', lattice=True, multiple_tables=True)

In [50]:
cover_crop_df = pdf_df(cover_crop)

<ipython-input-2-1ebbe04b62a3>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_acres2['Year'] = dataframe_acres2['Year'].astype('int')
<ipython-input-2-1ebbe04b62a3>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_acres2['Corn'] = [float(str(i).replace(",", "")) for i in dataframe_acres2['Corn']]
<ipython-input-2-1ebbe04b62a3>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [51]:
cover_crop_2012 = cover_crop_df.loc[cover_crop_df['Year'] == 2011, 'Corn'].array + observed_change_timeperiod(cover_crop_df)
cover_crop_df = cover_crop_df.append({'Year':2012,'Corn':cover_crop_2012[0], 'Corn (Thousands of acres)':(cover_crop_2012[0]/1000)}, ignore_index=True)
cover_crop_df.sort_values('Year', inplace=True, ignore_index=True)
cover_crop_df.columns = ['Year', 'Corn', 'IN Corn Cover Crop (Thousands of acres)']

In [52]:
cover_crop_final = cover_crop_df.loc[:, ['Year', 'IN Corn Cover Crop (Thousands of acres)']]

In [53]:
no_till_df = pdf_df(no_till)

<ipython-input-2-1ebbe04b62a3>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_acres2['Year'] = dataframe_acres2['Year'].astype('int')
<ipython-input-2-1ebbe04b62a3>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_acres2['Corn'] = [float(str(i).replace(",", "")) for i in dataframe_acres2['Corn']]
<ipython-input-2-1ebbe04b62a3>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [54]:
missing_df = create_missing_practices(no_till_df, no_till_df['Year'], no_till_df['Corn'])

In [55]:
parsed_till = parse_table(no_till_df,missing_df)
parsed_till.pop('Corn (Thousands of acres)')
parsed_till['Total Corn Acres'] = calc_total_land_change(parsed_till)
parsed_till['Total Corn Acres'] = round(parsed_till['Total Corn Acres'],)

In [56]:
year_till = parsed_till.loc[:, ['Year','Total Corn Acres']]
year_till['IN No Till Corn (Thousands of acres)'] = year_till['Total Corn Acres']/1000
final_no_till = year_till.loc[:, ['Year','IN No Till Corn (Thousands of acres)']]

In [57]:
crop_practices_df = final_no_till.merge(cover_crop_final,how='left')
crop_practices_df.fillna(0, inplace=True)
final_crop_practices = crop_practices_df[(crop_practices_df['Year'] < 2018)]

Read and create the dataframes for US Corn acerage, US Total acerage, and IN Corn acerage.

In [58]:
filename = input('Provide file name: ')
data = f'/Users/coleromanyk/Downloads/{filename}.csv'
US_corn_raw = pd.read_csv(data)
US_corn_raw = US_corn_raw.reset_index(drop=True)
US_corn_raw['Year'] = US_corn_raw.iloc[:,1].astype('int')
US_corn_raw['Total US Corn Acres (Thousands of acres)'] = US_corn_raw.iloc[:,19].str.replace(",", "").astype('float')//1000

Provide file name: US Corn


In [59]:
US_corn = US_corn_raw.loc[:,['Year','Total US Corn Acres (Thousands of acres)']]

In [60]:
filename = input('Provide file name: ')
data = f'/Users/coleromanyk/Downloads/{filename}.csv'
IN_corn_raw = pd.read_csv(data)
IN_corn_raw = IN_corn_raw.reset_index(drop=True)
IN_corn_raw['Year'] = IN_corn_raw.iloc[:,1].astype('int')
IN_corn_raw['Total IN Corn Acres (Thousands of acres)'] = IN_corn_raw.iloc[:,19].str.replace(",", "").astype('float')//1000

Provide file name: IN Corn


In [61]:
IN_corn = IN_corn_raw.loc[:,['Year','Total IN Corn Acres (Thousands of acres)']]

In [62]:
filename = input('Provide file name: ')
data = f'/Users/coleromanyk/Downloads/{filename}.csv'
US_total_raw = pd.read_csv(data,)
US_total_raw = US_total_raw.T
US_total_raw.iloc[0,0] = 'Year'
US_total_raw.iloc[0,1] = 'US Total (Thousands of acres)'
US_total_raw.reset_index(drop=True,inplace=True)
US_total_raw.columns = US_total_raw.iloc[0,:]
US_year_total = US_total_raw.loc[1:,['Year','US Total (Thousands of acres)']].reset_index(drop=True)
US_year_total['Year'] = US_year_total['Year'].astype('int')
US_year_total['US Total (Thousands of acres)'] = US_year_total['US Total (Thousands of acres)'].str.replace(",", "").astype('int')

Provide file name: US Total


In [63]:
missing_US_total = create_missing_df(US_year_total, US_year_total['Year'], US_year_total['US Total (Thousands of acres)'])
parse_US_total = parse_table(US_year_total,missing_US_total)
parse_US_total['US Total (Thousands of acres)'] = calc_total_land_change(parse_US_total)
parse_US_total['US Total (Thousands of acres)'] = round(parse_US_total['US Total (Thousands of acres)'],)
US_total = parse_US_total.loc[:,['Year','US Total (Thousands of acres)']]
final_US_total = US_total[(US_total['Year'] >= 1990)]

Merge the the acerage dfs with the practices dfs to create acres_and_practices df on year.

In [64]:
acres_df = final_US_total.merge(US_corn)
final_acres = acres_df.merge(IN_corn)
acres_and_practices = final_acres.merge(final_crop_practices)

Merge all of the data into a single dataframe Cleaned_data on year.

In [65]:
Cleaned_data = Emissions_landuse_fertilizer.merge(acres_and_practices)

In [66]:
Cleaned_data['Emissions per Acre of Corn IN (MMT CO2 Eq.)'] = (Cleaned_data['Total (MMT CO2 Eq.)']*Cleaned_data['Total IN Corn Acres (Thousands of acres)'])/Cleaned_data['Total US Corn Acres (Thousands of acres)']

Create the IN_emission_data_clean df. 1st calculate the emissions in MMT CO2 eq. per acre of corn in IN. Then join only the appropriate series to the final table.

In [67]:
years = Cleaned_data.iloc[:,0]
IN_emissions = Cleaned_data.iloc[:,-1]
IN_land_fert = Cleaned_data.iloc[:,2:7]
IN_practices = Cleaned_data.iloc[:,9:12]
IN_emissions_data_clean = pd.DataFrame(years)
IN_emissions_data_clean = IN_emissions_data_clean.join(IN_emissions,how='left') 
IN_emissions_data_clean = IN_emissions_data_clean.join(IN_land_fert,how='left')
IN_emissions_data_clean = IN_emissions_data_clean.join(IN_practices,how='left')

In [68]:
IN_emissions_data_clean

,Year,Emissions per Acre of Corn IN (MMT CO2 Eq.),Total Crop Land (Thousands of acres),Total Dev Land (Thousands of acres),N,PO4,Potash,Total IN Corn Acres (Thousands of acres),IN No Till Corn (Thousands of acres),IN Corn Cover Crop (Thousands of acres)
0,1990,23.7426,13686.96,1967.36,139,75,111,5450.0,479.255,0.000
1,1991,22.8044,13618.98,1988.58,135,78,112,5550.0,723.426,0.000
2,1992,23.2971,13551.00,2009.80,143,66,107,5970.0,967.598,0.000
3,1993,24.523,13527.82,2047.96,134,68,114,5400.0,1211.769,0.000
4,1994,23.7204,13504.64,2086.12,147,74,112,5960.0,1131.817,0.000
5,1995,23.7421,13481.46,2124.28,132,68,107,5300.0,1051.866,0.000
6,1996,22.6519,13458.28,2162.44,138,64,110,5450.0,971.914,0.000
7,1997,23.1483,13435.10,2200.60,146,72,107,5750.0,891.962,0.000
8,1998,22.4589,13418.76,2228.64,146,61,118,5550.0,968.033,0.000
9,1999,23.175,13402.42,2256.68,154,56,116,5670.0,1044.103,0.000
